In [1]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [2]:
import pprint
pp = pprint.PrettyPrinter()

In [51]:
scope = ['https://spreadsheets.google.com/feeds']
creds = ServiceAccountCredentials.from_json_keyfile_name('../sheets_backend_secret.json', scope)
client = gspread.authorize(creds)

In [52]:
sheet = client.open('Summary_of_de_novo_assembly_stats').worksheet('Summary')
labels = sheet.range('A4:AH4')
pp.pprint([x.value for x in labels])


['',
 'F. angustifolia subsp. angustifolia (FRAX01)',
 'F. apertisquamifera (FRAX02)',
 'F. caroliniana (FRAX03)',
 'F. dipetala (FRAX04)',
 'F. latifolia (FRAX05)',
 'F. mandshurica (FRAX06)',
 'F. ornus (FRAX07)',
 'F. paxiana (FRAX08)',
 'F. pennsylvannica (FRAX09)',
 'F. pennsylvannica (FRAX10)',
 'F. quadrangulata (FRAX11)',
 'F. sieboldiana (FRAX12)',
 'F. velutina (FRAX13)',
 'F. americana (FRAX14)',
 'F. angustifolia subsp. oxycarpa  (FRAX15)',
 'F. angustifolia subsp. syriaca (FRAX16)',
 '',
 '',
 'F. goodingii (FRAX19)',
 'F. greggii (FRAX20)',
 'F. griffithii (FRAX21)',
 'F. nigra (FRAX23)',
 '',
 'F. xanthoxyloides (FRAX25)',
 'F. albicans (FRAX26)',
 'F. anomala (FRAX27)',
 'F. baroniana (FRAX28)',
 'F. bungeana (FRAX29)',
 'F. chinensis [subsp. chinensis?] (FRAX30)',
 'F. cuspidata  (FRAX31)',
 'F. floribunda  (FRAX32)',
 'F. platypoda (FRAX33)',
 'F. uhdei (FRAX34)']


# Assembleathon Conversion

In [55]:
import re
import os
BASE_DIR = 'D:\\josiah\\Documents\\Research\\Thesis - Genome Symmetry\\data\\Assembleathon\\'

def format_assemblethon_file(filename):
    with open (filename, 'r' ) as f:
        stats_file = f.read()
    stats_file = re.sub(' {2,}', r'\t', stats_file, flags = re.M)
    stats_file = re.sub('^\t', r'', stats_file, flags = re.M)
    return stats_file
print(format_assemblethon_file('FRAX07_CLC_SSPACE_GAPCLOSER_ASSEMBLETHON.txt'))

Number of scaffolds	417940
Total size of scaffolds	855147449
Total scaffold length as percentage of known genome size	760.1%
Longest scaffold	367815
Shortest scaffold	200
Number of scaffolds > 500 nt	168610	40.3%
Number of scaffolds > 1K nt	77211	18.5%
Number of scaffolds > 10K nt	20212	4.8%
Number of scaffolds > 100K nt	251	0.1%
Number of scaffolds > 1M nt	0	0.0%
Mean scaffold size	2046
Median scaffold size	425
N50 scaffold length	21026
L50 scaffold count	10470
NG50 scaffold length	82739
LG50 scaffold count	528
N50 scaffold - NG50 scaffold length difference	61713
scaffold %A	28.32
scaffold %C	14.99
scaffold %G	14.98
scaffold %T	28.32
scaffold %N	13.39
scaffold %non-ACGTN	0.00
Number of scaffold non-ACGTN nt	0

Percentage of assembly in scaffolded contigs	66.8%
Percentage of assembly in unscaffolded contigs	33.2%
Average number of contigs per scaffold	1.1
Average length of break (>25 Ns) between contigs in scaffold	274

Number of contigs	470798
Number of contigs in scaffolds	78758
Numb

In [14]:
labels_of_interest = ['N50 scaffold length', 'L50 scaffold count', 'Number of contigs', 'Number of scaffolds','Longest contig']

In [53]:
def grab_values_from_file(stats_file):
    values = []
    for line in stats_file.split('\n'):
        parts = line.split('\t')
        if parts[0] in labels_of_interest:
            print(parts[0], parts[1])
            values.append(parts[1])
    return values
grab_values_from_file(stats_file)

Number of scaffolds 417940
N50 scaffold length 21026
L50 scaffold count 10470
Number of contigs 470798
Longest contig 342910


['417940', '21026', '10470', '470798', '342910']

In [49]:
def matching_col(frax_number):
    a = [v for v in labels if frax_number in v.value]
    print(a)
    return a[0].col  # gspread.utils.rowcol_to_a1(23, a[0].col)
matching_col('FRAX07')

[<Cell R4C8 'F. ornus (FRAX07)'>]


8

In [60]:
def update_stats_for_FRAX_number(frax_number):
    print(">>> Updating Stats for ", frax_number)
    filename = os.path.join(BASE_DIR, frax_number + '_CLC_SSPACE_GAPCLOSER_ASSEMBLETHON.txt')
    stats_file = format_assemblethon_file(filename)
    values = grab_values_from_file(stats_file)
    my_column = matching_col(frax_number)
    for label_index, label in enumerate(labels_of_interest):
        sheet.update_cell(23 + label_index, 1, label)
        sheet.update_cell(23 + label_index, my_column, values[label_index])

In [61]:
update_stats_for_FRAX_number('FRAX06')

>>> Updating Stats for  FRAX06
Number of scaffolds 297505
N50 scaffold length 30144
L50 scaffold count 7359
Number of contigs 352183
Longest contig 272010
[<Cell R4C7 'F. mandshurica (FRAX06)'>]


In [62]:
def main():
    I_have_stats_for = 'FRAX06 FRAX07 FRAX09 FRAX11 FRAX14 FRAX15 FRAX16 FRAX19 FRAX20 FRAX21 FRAX23 FRAX25 FRAX26 FRAX27 FRAX28 FRAX29 FRAX30 FRAX31 FRAX32 FRAX33 FRAX34'.split()
    for frax_number in I_have_stats_for:
        update_stats_for_FRAX_number(frax_number)
main()

>>> Updating Stats for  FRAX06
Number of scaffolds 297505
N50 scaffold length 30144
L50 scaffold count 7359
Number of contigs 352183
Longest contig 272010
[<Cell R4C7 'F. mandshurica (FRAX06)'>]
>>> Updating Stats for  FRAX07
Number of scaffolds 417940
N50 scaffold length 21026
L50 scaffold count 10470
Number of contigs 470798
Longest contig 342910
[<Cell R4C8 'F. ornus (FRAX07)'>]
>>> Updating Stats for  FRAX09
Number of scaffolds 555484
N50 scaffold length 18659
L50 scaffold count 11707
Number of contigs 616703
Longest contig 480722
[<Cell R4C10 'F. pennsylvannica (FRAX09)'>]
>>> Updating Stats for  FRAX11
Number of scaffolds 175641
N50 scaffold length 50545
L50 scaffold count 3889
Number of contigs 211193
Longest contig 235128
[<Cell R4C12 'F. quadrangulata (FRAX11)'>]
>>> Updating Stats for  FRAX14
Number of scaffolds 357129
N50 scaffold length 4583
L50 scaffold count 38222
Number of contigs 357416
Longest contig 195469
[<Cell R4C15 'F. americana (FRAX14)'>]
>>> Updating Stats for 

# Scrap Examples

In [46]:
sheet.col_values(2)

['color', 'updating', 'failing', 'e', 'good', 'good', 'good']

In [ ]:
sheet.resize(rows=3, cols=4)

In [50]:
sheet.col_count, sheet.row_count

(6, 8)

In [44]:
sheet.append_row(['Doin', 'good'])

In [70]:
gspread.utils.rowcol_to_a1(sheet.row_count, sheet.col_count )

'F10'

In [67]:
cell_list = sheet.range('A1:' + gspread.utils.rowcol_to_a1(sheet.row_count, sheet.col_count ))

for cell in cell_list:
    cell.value = 'O_o'

# Update in batch
sheet.update_cells(cell_list)

In [38]:
sheet.update_cell(2,2, 'failing')

In [39]:
sheet.cell(2,2).value

'failing'